# [Accessing](viewINs.md)
<img src="../docs/web_euplotid/lab_meeting_slides_Accessing.png" style="width: 1000px;">

# 1 Pick what cell type and condition you want to look at
Using widgets in Jupyter we are able to dynamically access the annotated JSONs in the backend.

In [ ]:
#Run case
def update_run_name(change):
    tmp_run_names = [os.path.basename(c_type) for c_type in glob.glob("example_output/output_IN_graphs/" + change["new"] + "/*")]
    #add dummy variable to switch to so it renders
    (tmp_run_names).append(None)
    name_picker.options = tmp_run_names

def update_global_graph(change):
    if (change["new"] is not None):
        new_out_dir = "example_output/output_IN_graphs/" + cell_picker.value + "/" + change["new"] + "/"
        cur_global_graph = new_out_dir + change["new"] + "_global_graph_annotated.json"
        with open(cur_global_graph,"r") as json_comm:
            json_in = json.load(json_comm)
            draft_genome_out_annot = json_graph.node_link_graph(json_in)
        in_names_2json = dict()
        for node in draft_genome_out_annot.nodes(data=True):
            in_genes = node[1]["name"].split(" ")[1].split(",")
            in_names_2json[node[1]["name"].split(" ")[1]] = new_out_dir + node[1]["json_name"]
        sorted_in_names_2json = [(k, in_names_2json[k]) for k in sorted(in_names_2json, key=in_names_2json.get, reverse=True)]
        sorted_IN_jsons = OrderedDict(sorted_in_names_2json)
        in_picker.options = sorted_IN_jsons
        sad_table_wid.options = sorted_IN_jsons
        #Print attributes
        interact(in_json_out_plotly_fig, json_name=fixed(cur_global_graph),make_3D=fixed(True), global_in=fixed(True));
        print("Species: " + draft_genome_out_annot.graph["species"])
        print("Genome Version: " + draft_genome_out_annot.graph["genome_version"])
        print("Cell Type: " + draft_genome_out_annot.graph["tissue_type"])
    
def draw_sad_table(in_json_name): 
    out_dir = os.path.dirname(in_json_name) + "/"
    #Fetch the top SNP and max min for Genome browser for target IN
    with open(in_json_name,"r") as json_comm:
        json_in = json.load(json_comm)
        in_annot_graph = json_graph.node_link_graph(json_in)
        unserial_sad_table = pickle.loads(in_annot_graph.graph["sad_table"])
        for in_node in in_annot_graph.nodes(data=True):
            if "in_name" in in_node[1]:
                cur_in_name = in_node[1]["in_name"]
                target_in_name = in_node[1]["in_name"]
                cur_in_min = in_node[1]["in_min"]
                cur_in_max = in_node[1]["in_max"]
            if "top_open_snp" in in_node[1]:
                sad_mut = out_dir + in_node[1]["sad_mut_alt"]
                sad_ref = out_dir + in_node[1]["sad_mut_ref"]
                
                sad_all = out_dir + in_node[1]["sad_pdf"]
                chrom = in_node[1]["gen_chrom"]
                snp_loc = in_node[1]["gen_start"]
                for neigh in in_annot_graph[in_node[0]]:
                    open_region = neigh
    data = [
        go.Heatmap(
            z=unserial_sad_table.values,
            y=unserial_sad_table.index,
            x=unserial_sad_table.columns.values,
            colorscale="spectral",
        )
    ]

    layout = go.Layout(
        title='Basset SAD Prediction',
        xaxis = dict(ticks=''),
        yaxis = dict(ticks='' )
    )

    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)
    return (sad_ref, sad_mut)


cell_types = [os.path.basename(c_type) for c_type in glob.glob("example_output/output_IN_graphs/*")]
#picking cell type and run
in_args = dict()
cell_picker = widgets.Dropdown(
    options=cell_types,
    value = cell_types[0],
    description='Cell type:')
name_types = [os.path.basename(c_type) for c_type in glob.glob("example_output/output_IN_graphs/" + cell_types[0] + "/*")]
name_types.append(None)
name_picker = widgets.Dropdown(
    options=name_types,
    value = name_types[0],
    description='Run:')
in_picker = widgets.Dropdown(
    options=["No Run Picked"],
    description='Pick IN:')
sad_table_wid = widgets.Dropdown(
    options=["NA"],
    description="SAD Table")

cell_picker.observe(update_run_name, 'value')
name_picker.observe(update_global_graph, 'value')
#in_picker.observe(sad_table_wid, "value")
#inlink = widgets.jslink((in_picker, 'value'), (sad_table_wid, 'value'))
display(cell_picker)
display(name_picker)

# 2 Pick annotated Insulated Neighborhood (IN) to load up as defined by Genes within it

# 3 View IN in 1D on UCSC genome browser, the left and right most nodes define the boundaries

# 4 Annotated IN of picked Gene(s) with:
* eQTL in 32 tissues (FastQTL+GTeX)
* Ease of CRISPR editing (GT-Scan2)
* Results of over 2,000 GWAS studies (GRASP)
* 667 Methylomes (NGSMethDB)
* Estimate of deleteriousness (CADD Score)
* Convolutional Neural Network prediction of TF identity

# 5 View DNA-DNA interactions in 2D on the fly with [Higlass.io](higlass.io)

In [ ]:
%%javascript

var config = {
  "editable": true,
  "zoomFixed": false,
  "trackSourceServers": [
    "http://higlass.io/api/v1"
  ],
  "exportViewUrl": "http://higlass.io/api/v1/viewconfs/",
  "views": [
    {
      "uid": "aa",
      "initialXDomain": [
        509161199.6251172,
        2590838800.374883
      ],
      "autocompleteSource": "http://higlass.io/api/v1/suggest/?d=OHJakQICQD6gTD7skx4EWA&",
      "genomePositionSearchBoxVisible": true,
      "chromInfoPath": "//s3.amazonaws.com/pkerp/data/hg19/chromSizes.tsv",
      "tracks": {
        "top": [
          {
            "type": "horizontal-gene-annotations",
            "height": 60,
            "tilesetUid": "OHJakQICQD6gTD7skx4EWA",
            "server": "http://higlass.io/api/v1",
            "position": "top",
            "uid": "OHJakQICQD6gTD7skx4EWA",
            "name": "Gene Annotations (hg19)",
            "options": {
              "name": "Gene Annotations (hg19)"
            },
            "maxWidth": 4294967296,
            "maxZoom": 22
          },
          {
            "chromInfoPath": "//s3.amazonaws.com/pkerp/data/hg19/chromSizes.tsv",
            "type": "horizontal-chromosome-labels",
            "position": "top",
            "name": "Chromosome Labels (hg19)",
            "height": 30,
            "uid": "N_w4XVIpTPSpq2xBVTToLA",
            "options": {}
          }
        ],
        "left": [
          {
            "type": "vertical-gene-annotations",
            "width": 60,
            "tilesetUid": "OHJakQICQD6gTD7skx4EWA",
            "server": "http://higlass.io/api/v1",
            "position": "left",
            "name": "Gene Annotations (hg19)",
            "options": {
              "labelPosition": "bottomRight",
              "name": "Gene Annotations (hg19)"
            },
            "uid": "BtHSrxzrSUy21KQ7D6lmZA",
            "maxWidth": 4294967296,
            "maxZoom": 22
          },
          {
            "chromInfoPath": "//s3.amazonaws.com/pkerp/data/hg19/chromSizes.tsv",
            "type": "vertical-chromosome-labels",
            "position": "left",
            "name": "Chromosome Labels (hg19)",
            "width": 30,
            "uid": "V74xYbctSbKB2XSesy2Glg",
            "options": {}
          }
        ],
        "center": [
          {
            "uid": "c1",
            "type": "combined",
            "height": 200,
            "contents": [
              {
                "server": "http://higlass.io/api/v1",
                "tilesetUid": "CQMd6V_cRw6iCI_-Unl3PQ",
                "type": "heatmap",
                "position": "center",
                "options": {
                  "colorRange": [
                    "#FFFFFF",
                    "#F8E71C",
                    "#F5A623",
                    "#D0021B"
                  ],
                  "maxZoom": null,
                  "labelPosition": "bottomRight",
                  "name": "Rao et al. (2014) GM12878 MboI (allreps) 1kb"
                },
                "uid": "QGf8KgUsSPCoVlk-EPWn3w",
                "name": "Rao et al. (2014) GM12878 MboI (allreps) 1kb",
                "maxWidth": 4194304000,
                "binsPerDimension": 256,
                "maxZoom": 14
              }
            ],
            "position": "center",
            "options": {}
          }
        ],
        "right": [],
        "bottom": []
      },
      "initialYDomain": [
        619564198.6879103,
        2480435801.312089
      ],
      "layout": {
        "w": 6,
        "h": 12,
        "x": 0,
        "y": 0,
        "i": "aa",
        "moved": false,
        "static": false
      }
    }
  ],
  "zoomLocks": {
    "locksByViewUid": {},
    "locksDict": {}
  },
  "locationLocks": {
    "locksByViewUid": {},
    "locksDict": {}
  }
};

element.append('<div><div id="higlass" class="full-dim"></div></div>');

requirejs.config({
    paths: {
        "hglib": "//higlass.io/assets/scripts-third-party/hglib"
    },
    waitSeconds: 2
});

require(['hglib'], function(hglib) {
    hglib.createHgComponent( 
        document.querySelector("#higlass"), 
        config, 
        { bounded: true}, 
        function (api) { window.higlassApi = api; }
    );
});

# 6 All SNPs/CNVs w/in IN are fed through Basset for prediction of impact on  chromatin accesibility in 128 cell types

# 7 Rough global view derived from polymer simulation using [HSA](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0896-1)

# 8 In-silico mutation analysis of chromatin accessibility peak using [Basset](http://genome.cshlp.org/content/early/2016/06/10/gr.200535.115.abstract)

# 9 Virtual reality visualization of Annotated Insulated Neighborhood

# Local Deployment of Euplotid
[**INSTALL DOCKER HERE**](https://www.docker.com/community-edition#/download")
Then open your terminal or cmd and:
~~~
docker run --name euplotid -p 8890:8890 -tid \
	-v "/your/input/directory:/input_dir" \
	-v "/your/temporary/directory/:/tmp_dir" \
	-v "/your/output/directory/:/output_dir" \
	-v "/your/annotation/directory/:/annotation_dir" \
	dborgesr/euplotid:euplotid
~~~